<a href="https://colab.research.google.com/github/Robjects-Collaborations/WhatIsMyAdaptor/blob/main/find_compatible_adapters_type_c_only.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This testing system:

1. Tests Data Loading:
- Verifies CSV files are loaded correctly
- Checks for required columns
- Validates data structure

2. Tests Feature Compatibility:
- Checks connector type matching
- Verifies power delivery compatibility
- Tests display support matching

3. Tests Scoring System:
- Calculates scores for all adapters
- Verifies score ranges
- Checks scoring distribution

4. Tests Recommendations:
- Generates top 5 recommendations
- Verifies recommendation quality
- Checks score distribution

5. Provides Detailed Results:
- Shows test outcomes
- Displays compatibility scores
- Lists top recommendations

To use:


In [ ]:

import pandas as pd
import numpy as np
from datetime import datetime

class AdapterSystemTester:
    def __init__(self, adapter_csv, requirements_csv):
        self.adapter_db = pd.read_csv(adapter_csv)
        self.requirements = pd.read_csv(requirements_csv)
        self.test_results = {}

    def run_full_system_test(self):
        """Run comprehensive system tests"""
        print("Starting Full System Test\n")

        # Test 1: Data Loading
        self.test_data_loading()

        # Test 2: Feature Compatibility
        self.test_feature_compatibility()

        # Test 3: Scoring System
        self.test_scoring_system()

        # Test 4: Recommendations
        self.test_recommendations()

        print("\nSystem Test Complete")
        return self.test_results

    def test_data_loading(self):
        """Test data loading and structure"""
        print("Testing Data Loading...")

        # Check adapter database
        adapter_test = {
            'database_loaded': not self.adapter_db.empty,
            'required_columns': all(col in self.adapter_db.columns for col in [
                'connector_type_a',
                'connector_type_b',
                'supports_usb_pd',
                'max_power_delivery',
                'usb_version'
            ])
        }

        # Check requirements data
        requirements_test = {
            'requirements_loaded': not self.requirements.empty,
            'required_columns': all(col in self.requirements.columns for col in [
                'connector_type_a',
                'connector_type_b',
                'supports_usb_pd'
            ])
        }

        self.test_results['data_loading'] = {
            'adapter_database': adapter_test,
            'requirements': requirements_test
        }

        print(f"Database Loaded: {adapter_test['database_loaded']}")
        print(f"Requirements Loaded: {requirements_test['requirements_loaded']}\n")

    def test_feature_compatibility(self):
        """Test feature compatibility matching"""
        print("Testing Feature Compatibility...")

        # Get sample requirement
        sample_req = self.requirements.iloc[0]

        # Test connector compatibility
        compatible_adapters = self.adapter_db[
            (self.adapter_db['connector_type_a'] == sample_req['connector_type_a']) &
            (self.adapter_db['connector_type_b'] == sample_req['connector_type_b'])
        ]

        compatibility_results = {
            'connector_match': len(compatible_adapters) > 0,
            'power_delivery': compatible_adapters['supports_usb_pd'].any(),
            'display_support': compatible_adapters['supports_displayport'].any()
        }

        self.test_results['feature_compatibility'] = compatibility_results

        print(f"Connector Compatibility: {compatibility_results['connector_match']}")
        print(f"Power Delivery Match: {compatibility_results['power_delivery']}\n")

    def test_scoring_system(self):
        """Test adapter scoring system"""
        print("Testing Scoring System...")

        def calculate_adapter_score(adapter, requirements):
            score = 0

            # Basic compatibility (30 points)
            if (adapter['connector_type_a'] == requirements['connector_type_a'] and
                adapter['connector_type_b'] == requirements['connector_type_b']):
                score += 30

                # Power delivery (20 points)
                if adapter['supports_usb_pd'] == requirements['supports_usb_pd']:
                    score += 20

                # Display support (15 points)
                if adapter['supports_displayport'] == requirements['supports_displayport']:
                    score += 15

                # Additional features (35 points)
                if adapter['supports_thunderbolt'] == requirements['supports_thunderbolt']:
                    score += 10
                if adapter['supports_quick_charge'] == requirements['supports_quick_charge']:
                    score += 10
                if adapter['supports_audio'] == requirements['supports_audio']:
                    score += 15

            return score

        # Test scoring with sample data
        sample_req = self.requirements.iloc[0]
        scored_adapters = self.adapter_db.copy()
        scored_adapters['compatibility_score'] = scored_adapters.apply(
            lambda x: calculate_adapter_score(x, sample_req), axis=1
        )

        scoring_results = {
            'max_score': scored_adapters['compatibility_score'].max(),
            'min_score': scored_adapters['compatibility_score'].min(),
            'avg_score': scored_adapters['compatibility_score'].mean()
        }

        self.test_results['scoring_system'] = scoring_results

        print(f"Maximum Score: {scoring_results['max_score']}")
        print(f"Average Score: {scoring_results['avg_score']:.2f}\n")

    def test_recommendations(self):
        """Test recommendation generation"""
        print("Testing Recommendation System...")

        # Get top 5 recommendations
        sample_req = self.requirements.iloc[0]
        recommendations = self.get_top_recommendations(sample_req)

        recommendation_results = {
            'num_recommendations': len(recommendations),
            'score_range': recommendations['compatibility_score'].max() -
                         recommendations['compatibility_score'].min(),
            'unique_recommendations': recommendations['Item Name'].nunique()
        }

        self.test_results['recommendations'] = recommendation_results

        print(f"Number of Recommendations: {recommendation_results['num_recommendations']}")
        print(f"Score Range: {recommendation_results['score_range']}\n")

        # Display top recommendations
        print("Top Recommendations:")
        for _, rec in recommendations.iterrows():
            print(f"- {rec['Item Name']}: Score {rec['compatibility_score']:.2f}")

    def get_top_recommendations(self, requirements, n=5):
        """Get top n adapter recommendations"""
        scored_adapters = self.adapter_db.copy()

        # Calculate scores
        scored_adapters['compatibility_score'] = scored_adapters.apply(
            lambda x: self.calculate_compatibility_score(x, requirements), axis=1
        )

        # Sort and get top n
        return scored_adapters.nlargest(n, 'compatibility_score')[
            ['Item Name', 'compatibility_score', 'connector_type_a', 'connector_type_b']
        ]

    def calculate_compatibility_score(self, adapter, requirements):
        """Calculate compatibility score for an adapter"""
        score = 0

        # Basic compatibility
        if (adapter['connector_type_a'] == requirements['connector_type_a'] and
            adapter['connector_type_b'] == requirements['connector_type_b']):
            score += 30

            # Power delivery
            if adapter['supports_usb_pd'] == requirements['supports_usb_pd']:
                score += 20

            # Display support
            if adapter['supports_displayport'] == requirements['supports_displayport']:
                score += 15

            # Additional features
            if adapter['supports_thunderbolt'] == requirements['supports_thunderbolt']:
                score += 10
            if adapter['supports_quick_charge'] == requirements['supports_quick_charge']:
                score += 10
            if adapter['supports_audio'] == requirements['supports_audio']:
                score += 15

        return score


In [ ]:
import pandas as pd
from datetime import datetime

# Create sample requirements that match available adapter configurations
sample_requirements = {
    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f'),
    'connector_type_a': 'USB-C',
    'connector_type_b': 'USB-A',
    'supports_usb_pd': True,
    'max_power_delivery': '90W',  # Matches one of the available adapters
    'usb_version': 'USB 3.1',
    'supports_displayport': True,
    'supports_hdmi': True,
    'supports_alt_mode': True,
    'supports_thunderbolt': False,
    'supports_quick_charge': True,
    'supports_audio': True
}

# Convert to DataFrame and save as CSV
sample_df = pd.DataFrame([sample_requirements])
sample_df.to_csv('sample_requirements.csv', index=False)

# Verify the requirements match available adapters
def verify_requirements_match():
    # Load both files
    adapters_df = pd.read_csv('Adapter-Type-C.csv')
    requirements_df = pd.read_csv('sample_requirements.csv')

    # Get the first requirement
    req = requirements_df.iloc[0]

    # Find matching adapters
    matches = adapters_df[
        (adapters_df['connector_type_a'] == req['connector_type_a']) &
        (adapters_df['connector_type_b'] == req['connector_type_b']) &
        (adapters_df['supports_usb_pd'] == req['supports_usb_pd']) &
        (adapters_df['supports_displayport'] == req['supports_displayport']) &
        (adapters_df['supports_hdmi'] == req['supports_hdmi']) &
        (adapters_df['supports_alt_mode'] == req['supports_alt_mode']) &
        (adapters_df['supports_quick_charge'] == req['supports_quick_charge']) &
        (adapters_df['supports_audio'] == req['supports_audio'])
    ]

    print(f"Number of matching adapters: {len(matches)}")
    if len(matches) > 0:
        print("\nMatching Adapters:")
        for _, adapter in matches.iterrows():
            print(f"- {adapter['connector_type_a']} to {adapter['connector_type_b']}, "
                  f"Power: {adapter['max_power_delivery']}, "
                  f"Category: {adapter['category_rating']}")

# Run verification
verify_requirements_match()

Number of matching adapters: 1

Matching Adapters:
- USB-C to USB-A, Power: 90W, Category: Premium



This testing framework helps ensure:
- Data integrity
- Scoring accuracy
- Recommendation quality
- System reliability
- Feature compatibility


This code defines a class AdapterSystemTester that tests the adapter recommendation system using the provided CSV files. It includes tests for data loading, feature compatibility, scoring system, and recommendations. The run_system_test function initializes the tester and runs the full system test, printing detailed results at the end.



The run_system_test function initializes the AdapterSystemTester with the provided CSV files and runs a full system test. The test results are then printed for each test category. The test_data_loading function checks if the data is loaded correctly, the test_feature_compatibility function checks feature compatibility, the test_scoring_system function tests the scoring system, and the test_recommendations function tests the recommendation generation.

You can run this script to test the adapter recommendation system using the provided CSV files. The test results will provide insights into the system's performance and accuracy.
